### Restart the failed us-east import

In [ ]:
!import.sh -o import -t 24 -l import/statements_live-us-east -json {\"waitMessageTimeoutSeconds\":300} -r us-east -b / -a -w 400000 -bulk > import-useast-live-statements-part3.log

After having started the import we can see that the number of threads decreases: 

 - first to 16: this makes sense since 8 partitions are empty
 - then to 8!!!

Running stream.sh

    > stream.sh lag -k --verbose -l import/statements_live-us-east
    
| partition | lag | pos | end | posOffset |?endOffset?|
| --- | ---: | ---: | ---: | ---: | ---: |
|All|120717538|424811048|545528586|14069052|22937322|
|0|8728788|14208534|22937322|14208534|22937322|
|1|8731460|14149276|22880736|14149276|22880736|
|2|8710766|14197138|22907904|14197138|22907904|
|3|8723144|14176024|22899168|14176024|22899168|
|4|8726814|14203752|22930566|14203752|22930566|
|5|8797547|14131063|22928610|14131063|22928610|
|6|8659158|14186244|22845402|14186244|22845402|
|7|8768832|14139036|22907868|14139036|22907868|
|8|8503380|14126196|22629576|14126196|22629576|
|9|8587516|14070506|22658022|14070506|22658022|
|10|8527171|14135177|22662348|14135177|22662348|
|11|8571420|14069052|22640472|14069052|22640472|
|12|4188615|18438921|22627536|18438921|22627536|
|13|4251018|18382104|22633122|18382104|22633122|
|14|4200629|18437197|22637826|18437197|22637826|
|15|4041280|18630620|22671900|18630620|22671900|
|16|0|22613778|22613778|22613778|22613778|
|17|0|22633500|22633500|22633500|22633500|
|18|0|22629768|22629768|22629768|22629768|
|19|0|22640196|22640196|22640196|22640196|
|20|0|22680138|22680138|22680138|22680138|
|21|0|22658220|22658220|22658220|22658220|
|22|0|22667322|22667322|22667322|22667322|
|23|0|22607286|22607286|22607286|22607286|
    

In the logs I can see a lot of rebalance:
    
    Rollback current batch because of consumer rebalancing
    
    Incomplete rebalance during poll, raising exception, revoked: true, lost: false

Looking at datadog the number of threads decreases regularly and the throughput also goes down:

<img src="monitoring/import-us-east-threads.png"/>

Doing one more stream.sh lag -k

| partition | lag | pos | end | posOffset |?endOffset?|
| --- | ---: | ---: | ---: | ---: | ---: |
|All|115319538|430209048|545528586|14385480|22937322|
|0|8412344|14524978|22937322|14524978|22937322|
|1|8403451|14477285|22880736|14477285|22880736|
|2|8391416|14516488|22907904|14516488|22907904|
|3|8393604|14505564|22899168|14505564|22899168|
|4|8399592|14530974|22930566|14530974|22930566|
|5|8472783|14455827|22928610|14455827|22928610|
|6|8334624|14510778|22845402|14510778|22845402|
|7|8442882|14464986|22907868|14464986|22907868|
|8|8176716|14452860|22629576|14452860|22629576|
|9|8272542|14385480|22658022|14385480|22658022|
|10|8202332|14460016|22662348|14460016|22662348|
|11|8247576|14392896|22640472|14392896|22640472|
|12|3820467|18807069|22627536|18807069|22627536|
|13|3886918|18746204|22633122|18746204|22633122|
|14|3843078|18794748|22637826|18794748|22637826|
|15|3619213|19052687|22671900|19052687|22671900|
|16|0|22613778|22613778|22613778|22613778|
|17|0|22633500|22633500|22633500|22633500|
|18|0|22629768|22629768|22629768|22629768|
|19|0|22640196|22640196|22640196|22640196|
|20|0|22680138|22680138|22680138|22680138|
|21|0|22658220|22658220|22658220|22658220|
|22|0|22667322|22667322|22667322|22667322|
|23|0|22607286|22607286|22607286|22607286|

So, it seems that there was progres on each of the partitions ...

All the nodes were stopped and restart to update the configuration.
The import was anyway running with only 2 threads at about 3,000 docs/s.

| partition | lag | pos | end | posOffset |?endOffset?|
| --- | ---: | ---: | ---: | ---: | ---: |
|All|91792038|453736548|545528586|15391176|22937322|
|0|7400256|15537066|22937322|15537066|22937322|
|1|7393776|15486960|22880736|15486960|22880736|
|2|7378962|15528942|22907904|15528942|22907904|
|3|7385125|15514043|22899168|15514043|22899168|
|4|7387846|15542720|22930566|15542720|22930566|
|5|7467498|15461112|22928610|15461112|22928610|
|6|7323889|15521513|22845402|15521513|22845402|
|7|7433928|15473940|22907868|15473940|22907868|
|8|7173480|15456096|22629576|15456096|22629576|
|9|7266216|15391806|22658022|15391806|22658022|
|10|7207151|15455197|22662348|15455197|22662348|
|11|7249296|15391176|22640472|15391176|22640472|
|12|953107|21674429|22627536|21674429|22627536|
|13|1031907|21601215|22633122|21601215|22633122|
|14|979203|21658623|22637826|21658623|22637826|
|15|760398|21911502|22671900|21911502|22671900|
|16|0|22613778|22613778|22613778|22613778|
|17|0|22633500|22633500|22633500|22633500|
|18|0|22629768|22629768|22629768|22629768|
|19|0|22640196|22640196|22640196|22640196|
|20|0|22680138|22680138|22680138|22680138|
|21|0|22658220|22658220|22658220|22658220|
|22|0|22667322|22667322|22667322|22667322|
|23|0|22607286|22607286|22607286|22607286|


In [ ]:
!import.sh -o import -t 24 -l import/statements_live-us-east -json {\"waitMessageTimeoutSeconds\":600\,\"batchThresholdS\":20\,\"retryDelayS\":10} -r us-east -b / -a -w 400000 -bulk > import-useast-live-statements-part4.log

In [1]:
# restart the import after having changed the balancing 
!import.sh -o import -t 24 -l import/statements_live-us-east -r us-east -b / -a -w 400000 -bulk > import-useast-live-statements-RoundRobinAssignor.log

In [7]:
!import.sh -o import -t 24 -batchSize 1 -l import/statements_live-us-east -r us-east -b / -a -w 400000 -bulk 

Using config /nxbench/notebooks/11B-Steps/nuxeo.properties
url=http://127.0.0.1:8080/nuxeo
login=nco-admin
Nuxeo Client configured
 (async mode actiavted)
Connected to Nuxeo Server 11.3.26
Running Operation:StreamImporter.runDocumentConsumersEx
   nbThreads: 24 
   blockPostCommitListeners: true 
   logName: import/statements_live-us-east 
   blockDefaultSyncListeners: true 
   blockAsyncListeners: true 
   rootFolder: / 
   logSize: 8 
   blockIndexing: true 
   batchSize: 1 
Async Automation Execution Scheduled
  => status url:[http://127.0.0.1:8080/nuxeo/site/api/v1/automation/StreamImporter.runDocumentConsumersEx/@async/60df2e00-347d-47b4-938e-ba9c648c02f4/status]
#####################
Execution completed

waiting for end of Async Exec
url=http://127.0.0.1:8080/nuxeo
login=nco-admin
Nuxeo Client configured
.
Running completed
elapsed:0.0
committed:0
failures:24
consumers:24
throughput:0.0

Exit after 120 s


The new import is quickly back to 2 threads, but the throughput seems to remain stable.

| partition | lag | pos | end | posOffset |?endOffset?|
| --- | ---: | ---: | ---: | ---: | ---: |
|All|4486945|541041641|545528586|21386951|22937322|
|0|1235731|21701591|22937322|21701591|22937322|
|1|1493785|21386951|22880736|21386951|22880736|
|2|0|22907904|22907904|22907904|22907904|
|3|0|22899168|22899168|22899168|22899168|
|4|0|22930566|22930566|22930566|22930566|
|5|0|22928610|22928610|22928610|22928610|
|6|19|22845383|22845402|22845383|22845402|
|7|21|22907847|22907868|22907847|22907868|
|8|0|22629576|22629576|22629576|22629576|
|9|0|22658022|22658022|22658022|22658022|
|10|963451|21698897|22662348|21698897|22662348|
|11|793926|21846546|22640472|21846546|22640472|
|12|0|22627536|22627536|22627536|22627536|
|13|6|22633116|22633122|22633116|22633122|
|14|6|22637820|22637826|22637820|22637826|
|15|0|22671900|22671900|22671900|22671900|
|16|0|22613778|22613778|22613778|22613778|
|17|0|22633500|22633500|22633500|22633500|
|18|0|22629768|22629768|22629768|22629768|
|19|0|22640196|22640196|22640196|22640196|
|20|0|22680138|22680138|22680138|22680138|
|21|0|22658220|22658220|22658220|22658220|
|22|0|22667322|22667322|22667322|22667322|
|23|0|22607286|22607286|22607286|22607286|


### Comparing with US-WEST Import

US-WEST import was properly ballanced, but also ended up with some errors because we generated statements that correspond to non existing accounts.

Looking at the us-west-statement stream:

    > stream.sh lag -k --verbose -l import/statements_live-us-west
| partition | lag | pos | end | posOffset |?endOffset?|
| --- | ---: | ---: | ---: | ---: | ---: |
|All|926|534463768|534464694|22151016|22480860|
|0|303|22428897|22429200|22428897|22429200|
|1|0|22470462|22470462|22470462|22470462|
|2|161|22446403|22446564|22446403|22446564|
|3|0|22432452|22432452|22432452|22432452|
|4|0|22449114|22449114|22449114|22449114|
|5|326|22480534|22480860|22480534|22480860|
|6|0|22426758|22426758|22426758|22426758|
|7|34|22424582|22424616|22424582|22424616|
|8|0|22193016|22193016|22193016|22193016|
|9|0|22161648|22161648|22161648|22161648|
|10|0|22166154|22166154|22166154|22166154|
|11|0|22164276|22164276|22164276|22164276|
|12|0|22190682|22190682|22190682|22190682|
|13|0|22166694|22166694|22166694|22166694|
|14|0|22196970|22196970|22196970|22196970|
|15|13|22204871|22204884|22204871|22204884|
|16|0|22191432|22191432|22191432|22191432|
|17|0|22205838|22205838|22205838|22205838|
|18|0|22160400|22160400|22160400|22160400|
|19|89|22175983|22176072|22175983|22176072|
|20|0|22167702|22167702|22167702|22167702|
|21|0|22151016|22151016|22151016|22151016|
|22|0|22183194|22183194|22183194|22183194|
|23|0|22224690|22224690|22224690|22224690|
    